<a href="https://colab.research.google.com/github/louisfb01/agent-course-notebooks/blob/main/notebooks/lesson_16_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 16: FastMCP — MCP Server and Client Quickstart

In this lesson, you will run a Model Context Protocol (MCP) server and MCP client using the FastMCP library, then explore how our research agent exposes MCP tools, MCP resources, and MCP prompts. We’ll start with a quick demo that runs the MCP client with an in-memory MCP server directly from this notebook, so you can get to try its capabilities immediately. Then, we’ll examine the MCP server and MCP client code structure.

Learning Objectives:
- Learn how to create an MCP server using `fastmcp`
- Learn how to create an MCP client using `fastmcp`
- Learn how to use the `fastmcp` library to expose MCP tools, MCP resources, and MCP prompts
- Learn how to use the `fastmcp` library to interact with an MCP server

## 1. Setup

Run the following command to install all the required packages to run this notebook.

In [ ]:
%pip install agentic-ai-engineering-course -q

### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions from the `Course Admin` lesson.

But here is a quick check on what you need to run this Notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  From the root of your project, run: `cp .env.example .env`
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:

In [ ]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

### Import Key Packages

In [ ]:
import nest_asyncio

nest_asyncio.apply()  # Allow nested async usage in notebooks

## 2. Try the agent (MCP client quickstart)

The research agent is made of an MCP server and an MCP client.

The MCP server is a `fastmcp` server that registers MCP tools, MCP resources, and MCP prompt via router modules. The MCP client is a `fastmcp` client that connects to the MCP server and allows you to interact with it, along with interacting with the LLM agent.

This quickstart runs the MCP client of the research agent inside the notebook kernel. It connects to the MCP server running in‑memory (same process), which is the only transport supported for running everything in the same notebook. So, we'll always run the MCP server in-memory in the notebooks.

Run the next code cell to start the MCP client. You will see some texts and can type commands directly in the input box that appears. The input box will be in different locations depending on where you are running the notebook from.

Once the client is running, you can type commands when prompted, such as:

- `/tools`: list all available MCP tools with names and descriptions.
- `/resources`: list all available MCP resources with their URIs.
- `/prompts`: list all available MCP prompts by name and description.
- `/prompt/full_research_instructions_prompt`: fetch the research workflow prompt and inject it into the conversation.
- `/resource/system://memory`: read and print the server memory stats (an example of running an MCP resource).
- `/model-thinking-switch`: toggle model “thinking” traces on/off. By default it is true, which means that you'll see the agent's thoughts in the conversation before each answer or tool call.
- Any other text: treated as a normal user message for the agent, which may use the MCP server tools for answering.
- `/quit`: terminate the client.

At first, try with the following commands and see what happens:
- `Hello! Who are you?`
- `/tools`
- `/resource/system://memory`
- `/quit`

In [ ]:
# Run the MCP client in-kernel
import sys

from research_agent_part_2.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup


# Start client with in-memory server
await run_client()

Whenever you want, you can run the previous cell again to try the client.

Now, let's see how the MCP server works.

## 3. MCP Server Overview

The purpose of this section is to show how the MCP server is created with `fastmcp` and how it wires MCP tools, MCP resources, and MCP prompts.

The MCP server is a `fastmcp` server that registers MCP tools (actions with side effects like scraping webpages, transcribing videos, etc.), MCP resources (read-only endpoints for information like system status or memory), and MCP prompts (reusable instruction blocks, such as our agent workflow) via router modules.

The MCP server follows a FastAPI‑like layout for clarity and scalability. It is structured as follows:

- `server.py`: Entry point exposing `create_mcp_server()` and a `__main__` runner.
- `routers/`: Functions that attach endpoints to the FastMCP instance.
  - `tools.py`: registers all MCP tools.
  - `resources.py`: registers all MCP resources.
  - `prompts.py`: registers all MCP prompts.
- `tools/`: MCP tools implementations.
- `resources/`: MCP resources implementations.
- `prompts/`: MCP prompts implementations (e.g. full workflow instructions for the agent).
- `app/`: Functions implementing business logic.
- `utils/`: Utility functions.
- `config/`: Pydantic settings (`settings.py`) for server name/version, logging, model choices, and API keys.

This separation keeps orchestration thin at the server boundary while allowing each capability (tool/resource/prompt) to evolve independently.

Let's see now how the MCP server is created.

Source:
_mcp_server/src/server.py_

```python
from fastmcp import FastMCP

from .config.settings import settings
from .routers.prompts import register_mcp_prompts
from .routers.resources import register_mcp_resources
from .routers.tools import register_mcp_tools


def create_mcp_server() -> FastMCP:
    """
    Create and configure the MCP server instance.

    This function can be imported to get a configured MCP server
    for use with in-memory transport in clients.

    Returns:
        FastMCP: Configured MCP server instance
    """
    # Create the FastMCP server instance
    mcp = FastMCP(
        name=settings.server_name,
        version=settings.version,
    )

    # Register all MCP endpoints
    register_mcp_tools(mcp)
    register_mcp_resources(mcp)
    register_mcp_prompts(mcp)

    return mcp
```

Notice how the `FastMCP` instance is created and how the `mcp` object is passed to the `register_mcp_tools`, `register_mcp_resources`, and `register_mcp_prompts` functions. It is pretty similar to how you would create a FastAPI app and attach endpoints to it!

### 3.1 Registering MCP Tools

Let's see now in particular how to register an MCP tool with `fastmcp`. This specific tool reads the article guidelines and extracts relevant references. Its implementation is in the `tools/extract_guidelines_urls_tool.py` file, along with other business logic functions in the `app/` folder. You can read the full file `mcp_server/src/routers/tools.py` to see all the 11 available MCP tools.

Source: _mcp_server/src/routers/tools.py_

```python
@mcp.tool()
async def extract_guidelines_urls(research_directory: str) -> Dict[str, Any]:
    """
    Extract URLs and local file references from article guidelines.

    Reads the ARTICLE_GUIDELINE_FILE file in the research directory and extracts:
    - GitHub URLs
    - Other HTTP/HTTPS URLs
    - Local file references (files mentioned in quotes with extensions)

    Results are saved to GUIDELINES_FILENAMES_FILE in the research directory.
    """
    result = extract_guidelines_urls_tool(research_directory)
    return result
```

This tool is the first step in the workflow. It reads the article guideline and writes a structured file containing URLs and local references. Notice how it requires a `research_directory` input, which is the path to the research directory containing a `article_guideline.md` file.


Let's test it with a sample article guideline. In the research agent folder, there's a `data/sample_research_folder` folder with an `article_guideline.md` file. Let's use it as input for the `extract_guidelines_urls` tool.


> **Colab users:** if you installed the package in Google Colab, you can pass this absolute path directly to the MCP client:
>
> `/usr/local/lib/python3.12/dist-packages/research_agent_part_2/data/sample_research_folder`



Here is how it is structured:

```md
## Global Context of the Lesson

...

## Lesson Outline

## Section 1: Introduction

...

## Section 2: Understanding why agents need tools

...

## Section N: Conclusion

...

## Article code

Links to code that will be used to support the article. Always prioritize this code over every other piece of code found in the sources:

- [Notebook 1](https://github.com/path/to/notebook.ipynb)

## Sources

- [Function calling with the Gemini API](https://ai.google.dev/gemini-api/docs/function-calling)
- [Function calling with OpenAI's API](https://platform.openai.com/docs/guides/function-calling)
- [Tool Calling Agent From Scratch](https://www.youtube.com/watch?v=ApoDzZP8_ck)
- [Efficient Tool Use with Chain-of-Abstraction Reasoning](https://arxiv.org/pdf/2401.17464v3)
- [Building AI Agents from scratch - Part 1: Tool use](https://www.newsletter.swirlai.com/p/building-ai-agents-from-scratch-part)
- [What is Tool Calling? Connecting LLMs to Your Data](https://www.youtube.com/watch?v=h8gMhXYAv1k)
- [ReAct vs Plan-and-Execute: A Practical Comparison of LLM Agent Patterns](https://dev.to/jamesli/react-vs-plan-and-execute-a-practical-comparison-of-llm-agent-patterns-4gh9)
- [Agentic Design Patterns Part 3, Tool Use](https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/)
```

Normally, an `article_guideline.md` file would contain detailed information about the article to write, including the outline, the sections, the sources, and the code, as the research agent needs this information to look for the best content to include in the article. In this sample file, we have a simplified version of an article guideline.

Now, run the next code cell to run the research agent MCP client again, and give it the following command. Make sure to replace the folder path with your actual absolute folder path, otherwise the tool will not find the file.
- Command to give to the client: `Run the "extract_guidelines_urls" tool with the "data/sample_research_folder" directory as research folder and stop after the tool has finished running.`.

If you are on **Colab**, you can instead pass the absolute path:
`/usr/local/lib/python3.12/dist-packages/research_agent_part_2/data/sample_research_folder`

In case you provide the wrong path, notice how the tool will return an error and how the agent will ask you to provide a valid path and how to proceed.

*Important*: the agent will manage every message starting with the `/` as a command, so, if you want to provide the folder path in a message, you need to write something like this: `Here is the folder path: /absolute/path/to/the/folder`.

In [ ]:
# Run the MCP client in-kernel
import sys

from research_agent_part_2.mcp_client.src.client import main as client_main


async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup


# Start client with in-memory server
await run_client()

Notice the agent's thoughts. If everything ran correctly, you'll see the text "Tool execution successful". If so, notice that there is a new folder named `.nova` in the research directory, with a file `guidelines_filenames.json` inside. This file contains the URLs and local references extracted from the article guideline.

Its content should be like this:

```json
{
  "github_urls": [
    "https://github.com/path/to/notebook.ipynb"
  ],
  "youtube_videos_urls": [
    "https://www.youtube.com/watch?v=ApoDzZP8_ck",
    "https://www.youtube.com/watch?v=h8gMhXYAv1k"
  ],
  "other_urls": [
    "https://ai.google.dev/gemini-api/docs/function-calling",
    "https://platform.openai.com/docs/guides/function-calling",
    "https://arxiv.org/pdf/2401.17464v3",
    "https://www.newsletter.swirlai.com/p/building-ai-agents-from-scratch-part",
    "https://dev.to/jamesli/react-vs-plan-and-execute-a-practical-comparison-of-llm-agent-patterns-4gh9",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/"
  ],
  "local_file_paths": []
}
```

So, the tool has extracted those URLs from the `article_guideline.md` file and categorized them into the groups you see above.

We can run the above tool also programmatically as follows. The output shows the result of running it from the local setup of the author of this notebook. To run it, update the path of the `research_folder` variable with your absolute path to the `sample_research_folder` folder.

In [ ]:
from research_agent_part_2.mcp_server.src.tools import extract_guidelines_urls_tool

# On Colab you can use this path "/usr/local/lib/python3.12/dist-packages/research_agent_part_2/data/sample_research_folder
research_folder = "/your/absolute/path/to/sample_research_folder"
extract_guidelines_urls_tool(research_folder=research_folder)

We'll comment the output of this tool in the next lesson. In the next lessons, we'll run each tool one by one like in the above code cell, so you can see the output of each tool and understand how the research agent works.

### 3.2 Registering MCP Resources

Let's see now how to register an MCP resource endpoint using `fastmcp`.

Source:
_lessons/research_agent_part_2/mcp_server/src/routers/resources.py_

```python
@mcp.resource("system://memory")
async def memory_usage() -> Dict[str, Any]:
    """Monitor memory usage of the server."""
    return await get_memory_usage_resource()
```

It's very similar to how tools are registered, except that the `@mcp.resource()` decorator is used instead of the `@mcp.tool()` decorator.

Let's now run the `get_memory_usage_resource` function to see the memory usage of the server.

In [ ]:
from research_agent_part_2.mcp_server.src.resources import get_memory_usage_resource

await get_memory_usage_resource()

This output is the same output that an MCP client would get if it uses this MCP resource.

*Important*: in the research agent MCP client, we have only implemented the use of tools by the agent LLM, but we could have implemented the use of resources as well. Most MCP clients do not support resources yet, but their support is increasing.

### 3.3 Registering MCP Prompts

This section shows how MCP prompts are implemented with `fastmcp`. This specific prompt defines the agentic workflow for the research agent.

Source:
_mcp_server/src/routers/prompts.py_

```python
@mcp.prompt()
async def full_research_instructions_prompt() -> str:
    """Complete Nova research agent workflow instructions."""
    return await _get_research_instructions()
```

The prompt content encodes the full workflow orchestration the agent should follow when started via a prompt.

In practice, MCP prompts are triggered by users from an MCP client, not by the agent LLM. When a user triggers an MCP prompt, the MCP client would retrieve that prompt and load it to instruct the LLM on how to run the available tools in sequence (and sometimes in parallel) according to the workflow described in it.

For reference, here is the full prompt content of the only MCP prompt implemented in the research agent, which is the `full_research_instructions_prompt` prompt.

In [ ]:
from research_agent_part_2.mcp_server.src.prompts import (
    full_research_instructions_prompt,
)

prompt = await full_research_instructions_prompt()
print(prompt)

This is the instruction block that defines the agentic workflow for the research agent. In the next lessons, we'll go through each step defined in the workflow, learn how it is implemented, and run it in isolation.

Let's now see how the MCP client works.

## 4. MCP Client Overview

Here is the MCP client's layout. It is structured as follows:

- `client.py`: CLI entry point. Parses `--transport`, creates the client (in‑memory or stdio), fetches capabilities, prints the startup banner, and runs the interactive loop.
- `settings.py`: Centralized Pydantic settings for API keys, model selection, logging, transport, and server paths.
- `utils/`: Helper modules used by `client.py`.

The MCP client can run with two transports:

- **in-memory**: The client imports the server factory (the `create_mcp_server` function from the `client.py` file) and instantiates the server inside the same Python process. This is fast, simple to debug, and is what we use in this notebook.
- **stdio**: The client launches the server as a separate process and communicates using the MCP stdio transport. This mirrors how external MCP clients (e.g., editors) connect to servers and provides process isolation.

Let's see how the code of the `client.py` file works.

Source: _mcp_client/src/client.py_

```python
if args.transport == "in-memory":
    ...
    from mcp_server.src.server import create_mcp_server
    mcp_server = create_mcp_server()
    mcp_client = Client(mcp_server)

elif args.transport == "stdio":
    config = {
        "mcpServers": {
            "research-agent": {
                "transport": "stdio",
                "command": "uv",
                "args": [
                    "--directory", str(settings.server_main_path),
                    "run", "-m", "src.server",
                    "--transport", "stdio",
                ],
            }
        }
    }
    mcp_client = Client(config)

# At startup
tools, resources, prompts = await get_capabilities_from_mcp_client(mcp_client)
print_startup_info(tools, resources, prompts)

async with mcp_client:
    while True:
        # Get user input
        user_input = input("👤 You: ").strip()
        ...

        # Parse input
        parsed_input = parse_user_input(user_input)
        ...

        # Dispatch handling
        await handle_user_message(parsed_input=parsed_input, ...)
        ...
```

It does the following:
1) Parse the `--transport` flag.
2) If in-memory, build a `Client` with the FastMCP server object. If stdio, pass a config that tells FastMCP how to exec the server via `uv`.
3) Query the MCP server for its capabilities (tools/resources/prompts) and print them.
4) Enter the interactive loop: read input, parse it, and dispatch handling.

The code above is run when the MCP client is started. If you remember from previous cells, when the MCP client is started, it prints the following information:

```
🛠️ Available tools: 11
📚 Available resources: 2
💬 Available prompts: 1

Available Commands: /tools, /resources, /prompts, /prompt/<name>, /resource/<uri>, /model-thinking-switch, /quit
```

But, how does the MCP client know how many tools, resources, and prompts are available? Let's see how the `get_capabilities_from_mcp_client` function works.

Source:
_mcp_client/src/utils/mcp_startup_utils.py_

```python
async def get_capabilities_from_mcp_client(client: Client) -> tuple[List, List, List]:
    """Get available capabilities."""
    async with client:
        tools = await client.list_tools()
        resources = await client.list_resources()
        prompts = await client.list_prompts()

    return tools, resources, prompts
```

As you can see, the MCP client object has a `list_tools`, `list_resources`, and `list_prompts` method that returns the list of tools, resources, and prompts respectively. These lists contain information about their names, descriptions, parameters, and so on.

We are now ready to learn how the MCP client parses the user input and how it handles the user messages.

### 4.1 Parsing Input and Commands

The client supports a small command language. Input can be either a command (starting with `/`) or a freeform user message.

Possible commands are:
- `/tools`, `/resources`, `/prompts`
- `/prompt/<name>` (e.g., `/prompt/full_research_instructions_prompt`)
- `/resource/<uri>` (e.g., `/resource/system://status`)
- `/model-thinking-switch`
- `/quit`

The `parse_user_input` function simply classifies the input (no side effects) and it returns a `ProcessedInput` with metadata. Here are some examples:

In [ ]:
from research_agent_part_2.mcp_client.src.utils.parse_message_utils import (
    parse_user_input,
)

processed_input = parse_user_input("/tools")
print(processed_input.input_type)

processed_input = parse_user_input("/resources")
print(processed_input.input_type)

processed_input = parse_user_input("/prompt/full_research_instructions_prompt")
print(processed_input.input_type, processed_input.prompt_name)

processed_input = parse_user_input("Hello, how are you?")
print(processed_input.input_type)

These processed inputs are then used to dispatch the correct handling.

The `handle_user_message` function orchestrates the conversation, calling the appropriate helper for the parsed command, or appending a normal message and running the agent loop.

Here are some examples. Let's first create the MCP server and client, and get the server capabilities (available tools, resources, and prompts).

In [ ]:
from fastmcp import Client
from research_agent_part_2.mcp_client.src.utils.handle_message_utils import (
    handle_user_message,
)
from research_agent_part_2.mcp_client.src.utils.mcp_startup_utils import (
    get_capabilities_from_mcp_client,
)
from research_agent_part_2.mcp_server.src.server import create_mcp_server

# Create the MCP server and client
mcp_server = create_mcp_server()
mcp_client = Client(mcp_server)

# Get the MCP server capabilities
tools, resources, prompts = await get_capabilities_from_mcp_client(mcp_client)

Now, let's parse the user input and handle the user message with the `handle_user_message` function. Here is an example with commands (i.e. messages starting with `/`):

In [ ]:
# Parse the user input
processed_input = parse_user_input("/resources")
conversation_history = []
response = await handle_user_message(
    processed_input,
    tools,
    resources,
    prompts,
    conversation_history,
    mcp_client,
    thinking_enabled=True,
)

The `handle_user_message` function is basically a router that calls the appropriate helper for the parsed message. It is defined in the `handle_message_utils.py` file, you can read it to learn more about it.

As previously explained, the `tools` object contains the list of tools registered in the MCP server, retrieved by the `list_tools` method. If the input is of type `COMMAND_INFO_TOOLS`, the `handle_command` function is called.

Source:
_mcp_client/src/utils/command_utils.py_

```python
def handle_command(processed_input: ProcessedInput, tools: List, resources: List, prompts: List):
    """Handle informational commands.

    This function only handles informational commands (COMMAND_INFO_* types).
    """
    if processed_input.input_type == InputType.COMMAND_INFO_TOOLS:
        print_header("🛠️  Available Tools")
        for i, tool in enumerate(tools, 1):
            print_item(tool.name, tool.description, i, Color.BRIGHT_WHITE, Color.YELLOW)
    ...
```

This function retrieves, from each tool, the name and description, and prints them in a pretty format.

All the tools are managed in a similar way.

If the input message is of type `NORMAL_MESSAGE`, the `handle_agent_loop` function is called instead, which manages the agent loop for tool execution. Let's see how it works.

Source:
_mcp_client/src/utils/handle_agent_loop_utils.py_

```python
async def handle_agent_loop(
    conversation_history: List[types.Content],
    tools: List,
    client: Client,
    thinking_enabled: bool,
):
    """Handle the agent loop for tool execution."""
    # Initialize LLM client
    llm_config = build_llm_config_with_tools(tools, thinking_enabled)
    llm_client = LLMClient(settings.model_id, llm_config)

    while True:
        print()
        # Call LLM with current conversation history
        response = await llm_client.generate_content(conversation_history)

        # Extract and display thoughts as separate message (only if enabled)
        if thinking_enabled:
            thoughts = extract_thought_summary(response)
            ...

        # Check for function calls
        function_call_info = extract_first_function_call(response)
        if function_call_info:
            name, args = function_call_info

            # Check if this is a tool call
            is_tool = any(tool.name == name for tool in tools)

            if is_tool:
                ...

                # Execute the tool via MCP server
                tool_result = await execute_tool(name, args, client)
                # Add tool result to conversation history
                tool_response = f"Tool '{name}' executed successfully. Result: {tool_result}"
                conversation_history.append(types.Content(role="user", parts=[types.Part(text=tool_response)]))
                ...
        else:
            # Extract final text response - this ends the ReAct loop
            final_text = extract_final_answer(response)
            conversation_history.append(response.candidates[0].content)
            ...
            break  # Exit the agent loop
```

This function is the main loop that manages the agent loop for tool execution. It initializes the LLM client, builds the LLM configuration with the tools, and then enters the agent loop.

The loop is structured as follows:

1) Call the LLM with the current conversation history.
2) Extract and display thoughts as separate message (only if enabled).
3) Check for function calls.
4) If there is a function call, check if it is a tool call.
5) If it is a tool call, execute the tool via MCP server.
6) Add the tool result to the conversation history.

The `LLMClient` class is simply a wrapper class that allows to generate content (or a function call) with an LLM, independently from the specific LLM provider. Right now it only implements Google Gemini as model, but it can be easily extended to other models. It is defined in the `llm_utils.py` file.

The `build_llm_config_with_tools` function builds the LLM configuration with the tools, it only works with Gemini for now. It is defined in the `llm_utils.py` file as well. Here's its code.

```python
def build_llm_config_with_tools(mcp_tools: List, thinking_enabled: bool = True) -> types.GenerateContentConfig:
    """Build Gemini config with all MCP tools converted to Gemini format."""
    gemini_tools = []

    for tool in mcp_tools:
        gemini_tool = types.Tool(
            function_declarations=[
                types.FunctionDeclaration(
                    name=tool.name,
                    description=tool.description,
                    parameters=tool.inputSchema,
                )
            ]
        )
        gemini_tools.append(gemini_tool)

    # Create thinking config dynamically based on current state
    thinking_config = types.ThinkingConfig(
        include_thoughts=thinking_enabled,
        thinking_budget=settings.thinking_budget,
    )

    return types.GenerateContentConfig(
        tools=gemini_tools,
        thinking_config=thinking_config,
        automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
    )
```

The code above basically instructions the LLM to leverage thinking (if enabled) with the specificed thinking budget (i.e. the maximum number of tokens the LLM can use to think) and to use the available tools from the MCP server.

The other functions from the `handle_agent_loop` function, like `extract_thought_summary` and `extract_final_answer`, are used to extract the thoughts and the final answer from the LLM response. It's boilerplate code that works for Gemini and can be copypasted for other projects.

The `execute_tool` function is used to execute the tool via MCP server. It is defined in the `handle_agent_loop_utils.py` file. Here's its code.

```python
async def execute_tool(name: str, args: dict, client: Client):
    """Execute a tool and return the result."""
    ...
    tool_result = await client.call_tool(name, args)
    return tool_result
```

It uses the `call_tool` method of the `Client` object to execute the tool.

We can now test the MCP client with a user message that involves tool execution and see how the agent behaves.

In [ ]:
# Parse the user input
path_to_research_folder = "/your/absolute/path/to/sample_research_folder"
message = (
    f"Call the 'extract_guidelines_urls' tool with the '{path_to_research_folder}' directory as research folder, and stop after the tool has finished running."
    "Don't run any other tool after the 'extract_guidelines_urls' tool has finished running."
    "If the tool fails, explain to me the error message."
)
processed_input = parse_user_input(message)
conversation_history = []
async with mcp_client:
    response = await handle_user_message(
        processed_input,
        tools,
        resources,
        prompts,
        conversation_history,
        mcp_client,
        thinking_enabled=True,
    )

We are good to go!

In the next lesson, we'll learn more about how the MCP prompt is used by the MCP client to orchestrate the agentic workflow.
Then, we'll go through each step of the research agent workflow, and we'll see how to run each tool in isolation.